# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770826400025                   -0.52    9.0         
  2   -2.772147158114       -2.88       -1.32    1.0    104ms
  3   -2.772170258682       -4.64       -2.43    1.0    122ms
  4   -2.772170646085       -6.41       -3.13    1.0    108ms
  5   -2.772170722928       -7.11       -4.41    2.0    124ms
  6   -2.772170722983      -10.26       -4.62    1.0    130ms
  7   -2.772170723012      -10.53       -5.63    1.0    113ms
  8   -2.772170723015      -11.56       -6.13    2.0    143ms
  9   -2.772170723015      -12.96       -7.22    1.0    115ms
 10   -2.772170723015      -14.35       -7.21    2.0    145ms
 11   -2.772170723015   +  -13.94       -8.27    1.0    117ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770820929266                   -0.52    9.0         
  2   -2.7

1.773558011461611

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770837743608                   -0.52    9.0         
  2   -2.772061961324       -2.91       -1.32    1.0    104ms
  3   -2.772083093942       -4.68       -2.45    1.0    139ms
  4   -2.772083343235       -6.60       -3.15    1.0    107ms
  5   -2.772083416471       -7.14       -3.96    2.0    136ms
  6   -2.772083417681       -8.92       -4.58    1.0    129ms
  7   -2.772083417810       -9.89       -5.46    2.0    125ms
  8   -2.772083417810      -12.56       -6.14    1.0    124ms
  9   -2.772083417811      -12.83       -6.77    2.0    137ms
 10   -2.772083417811   +  -13.89       -7.56    1.0    133ms
 11   -2.772083417811      -13.82       -8.66    1.0    116ms

Polarizability via ForwardDiff:       1.7725349681754452
Polarizability via finite difference: 1.773558011461611
